<a href="https://colab.research.google.com/github/pelagios/llm-lod-enriching-heritage/blob/main/notebooks/data_preparation/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing a Cultural Heritage Dataset for Named Entity Recognition

## Rationale

This recipe takes some source content and preprocesses it for use across the rest of the cookbook.

You can either:
- run queries to get sample content from the Cleveland Museum of Art API from a keyword (e.g. "Manet") or record ID search, or
- upload your own sample text (e.g. transcribed text from a digitised item).

It shows the necessary steps to process the results and format them into a JSON file that can then be fed into a NER process.

# Overview of the process

Most parts of the recipe simply need to be run in a Notebook environment like Colab, Binder, Jupyter Notebooks.

1. **Initialise the notebook**
- Run the first 'cells' to import the required libraries

2. **Fetch input**  
This is the part where you can provide some specific input.

You can:
   - Get artworks from the Cleveland Museum of Art API (by ID or keyword). You can run the code as-is, or play with changing options to search for specific keywords or record IDs. Or,
   - Upload your own text.

3. **Process text**  
   - Clean text - normalize Unicode, remove control characters, collapse spaces, mask URLs/emails.
   - Detect language - we use `langid.py` to assign a language code and confidence score.
   - Tokenize & split sentences - use spaCy's lightweight tokenizer and sentencizer to create tokens and sentence spans.  

4. **Assemble JSON**  
   - Combine results into a structured record:  
     - `text_original` and `text_clean`  
     - `language` (code + score)  
     - `sentences` (spans + text)  
     - `tokens` (spans + features)  
     - `meta` (counts + IDs)

# Step 1: Install the necessary packages and libraries

In [ ]:
%pip install spacy langid

In [ ]:
# pip install requests spacy langid

import re, unicodedata, json
from typing import List, Dict, Any, Tuple, Optional
import hashlib
import io

import requests
import spacy
import langid

try:
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

# Step 2: get content - run API queries or upload your own text

## Step 2a: Fetch Input from a Museum API

Get artworks from the Cleveland Museum of Art API (by ID or keyword).

In [ ]:
# ---------------------------
# 1) CMA API (single function)
# ---------------------------
def fetch_cma(source: str, *, mode: str = "search", limit: int = 100) -> List[Dict[str, Any]]: # the function outputs the first 100 results as a basic parameter
    """
    mode='id'     -> source is an artwork id
    mode='search' -> source is a keyword query
    Returns: list of {'id': <id>, 'text': <description or ''>}
    """
    base = "https://openaccess-api.clevelandart.org/api/artworks"

    if mode == "id":
        url = f"{base}/{source}"
        r = requests.get(url, timeout=20)
        if r.status_code == 404:
            # Graceful return instead of raising
            return []
        r.raise_for_status()
        data = r.json().get("data", {})
        return [{"id": data.get("id"), "text": (data.get("description") or "").strip()}]

    # search mode
    params = {"q": source, "skip": 0, "limit": limit}
    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    out = []
    for a in r.json().get("data", []):
        out.append({"id": a.get("id"), "text": (a.get("description") or "").strip()})
    return out

The code in the cells below will display a preview of the results for each function.

In [ ]:
# test the fetch_cma function and print an overview of the results, using the "search" mode and a keyword, e.g. "monet"
records = fetch_cma("monet", mode="search")
if not records:
    print("CMA by id: no record (likely 404).")

print(records[:10])

[{'id': 135382, 'text': "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of their home at Argenteuil. Her face is rendered in a radically bold Impressionist technique of mere daubs of paint quickly applied, just as the snow and trees are defined by broad, broken strokes of pure white and green."}, {'id': 136510, 'text': 'A skilled horticulturalist as well as an artist, Claude Monet spent the last 30 years of his life painting the private garden he designed and helped cultivate at his home in Giverny in northern France. The resultant canvases are notable for their varied motifs, formats, and sizes. Monumental in scale, this rendering of his water lily pond focuses on the momentary effects of sunlight as it both penetrates and reflects off its shimmering surface. By zeroing in on the water and omitting its horizon and surrounding banks, Monet infers a limitless expanse—a perception amplified by the painting’s vast horizontal format th

In [ ]:
# test the fetch_cma function and print an overview of the results, using the "id" mode and a sample record id.
records = fetch_cma("135382", mode="id")
if not records:
    print("CMA by id: no record (likely 404).")

print(records[:10])

[{'id': 135382, 'text': "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of their home at Argenteuil. Her face is rendered in a radically bold Impressionist technique of mere daubs of paint quickly applied, just as the snow and trees are defined by broad, broken strokes of pure white and green."}]


If you

# 2b: Using your own data

TODO: Add explanatory text here

In [ ]:
def fetch_local(source: Optional[str] = None):
  if IN_COLAB:
    # Upload a text file to use your own data
    uploaded = files.upload()
    records = [{"id": filename, "text": contents.decode("utf-8")} for filename, contents in uploaded.items()]
  else:
    # If running on your local machine, pass the location of the text file
    with open(source, "r") as f:
      records  = [{"id": source, "text": f.read().strip()}]

  return records

records = fetch_local()
records


Saving 7E59v2Ax.txt to 7E59v2Ax.txt
Saving 7E51v1x.txt to 7E51v1x (9).txt


[{'id': '7E59v2Ax.txt',
  'text': '\ufeff-----------------------------------------------------------------------------\r\nMAG X870E TOMAHAWK WIFI (MS-7E59) V2.A83 BIOS Release\r\n-----------------------------------------------------------------------------\r\n\r\n1. This is AMI BIOS release\r\n\r\n2. This BIOS fixes the following problem of the previous version:\r\n-  Improve memory compatibility.\r\n-  Optimized Secure Erase mechanism.\r\n\r\n3. 2025/08/01\r\n\r\n\r\n[Below information is Traditional Chinese language]\r\n\r\nA. AMI BIOS 正式發行\r\n\r\nB. 此版本修正下列問題:\r\n-  改善記憶體相容性問題。\r\n-  優化安全性清除機制。\r\n\r\nC. 更新日期: 公元2025年8月1號\r\n\r\n\r\n[Below information is Simplified Chinese language]\r\n\r\nA. AMI BIOS 正式发行\r\n\r\nB. 此版本修正下列问题:\r\n-  改善内存兼容性问题。\r\n-  优化安全性清除机制。\r\n\r\nC. 更新日期: 公元2025年8月1号'},
 {'id': '7E51v1x (9).txt',
  'text': '\ufeff----------------------------------------------------------------------------\r\nMAG X870 TOMAHAWK WIFI (MS-7E51) V1.A65 BIOS Release\r\n---------------

# Process text for NER

## Step 3a: Clean text

Normalize Unicode, remove control characters, collapse spaces, mask URLs/emails.  

In [ ]:
# ---------------------------
# 2) Pre-processing helpers
# ---------------------------
def clean_text(text: str) -> str:
    t = unicodedata.normalize("NFC", text)
    t = re.sub(r"[\u0000-\u0008\u000B\u000C\u000E-\u001F\u007F]", "", t)  # strip control chars
    t = re.sub(r"[ \t\u00A0]+", " ", t)                                   # collapse spaces
    t = re.sub(r"https?://\S+", "<URL>", t)                               # mask URLs
    t = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "<EMAIL>", t)               # mask emails
    return t

In [ ]:
# call the function clean text on the records output from the previous function, and print the first results.
clean_records = [clean_text(r["text"]) for r in records]
print(clean_records[:10])

['\ufeff-----------------------------------------------------------------------------\r\nMAG X870E TOMAHAWK WIFI (MS-7E59) V2.A83 BIOS Release\r\n-----------------------------------------------------------------------------\r\n\r\n1. This is AMI BIOS release\r\n\r\n2. This BIOS fixes the following problem of the previous version:\r\n- Improve memory compatibility.\r\n- Optimized Secure Erase mechanism.\r\n\r\n3. 2025/08/01\r\n\r\n\r\n[Below information is Traditional Chinese language]\r\n\r\nA. AMI BIOS 正式發行\r\n\r\nB. 此版本修正下列問題:\r\n- 改善記憶體相容性問題。\r\n- 優化安全性清除機制。\r\n\r\nC. 更新日期: 公元2025年8月1號\r\n\r\n\r\n[Below information is Simplified Chinese language]\r\n\r\nA. AMI BIOS 正式发行\r\n\r\nB. 此版本修正下列问题:\r\n- 改善内存兼容性问题。\r\n- 优化安全性清除机制。\r\n\r\nC. 更新日期: 公元2025年8月1号', '\ufeff----------------------------------------------------------------------------\r\nMAG X870 TOMAHAWK WIFI (MS-7E51) V1.A65 BIOS Release\r\n----------------------------------------------------------------------------\r\n\r\n1. This 

Then, detect the language of the dataset using the SpaCy langid module. This will help define the appropriate sentencizer for the following step.

In [ ]:
# detect language using langid

def detect_language(text: str) -> Dict[str, Any]:
    if not text:
        return {"language": "xx", "score": None}
    code, score = langid.classify(text)  # score = log-likelihood
    return {"language": code, "score": float(score)}

In [ ]:
# call the function detect_language and print the language id for the records output
lang_id = [detect_language(r) for r in clean_records]
print(lang_id[:10])

[{'language': 'zh', 'score': -1687.8875234127045}, {'language': 'zh', 'score': -1590.8130090236664}]


## Step 3b - Preprocess the texts

Using the Spacy sentencizer, based on the language detected by langid, split the output into sentences.

Then, assemble the structured record as a JSON output.

- Output fields:

     - `text_original` and `text_clean`  
     - `language` (code + score)  
     - `sentences` (spans + text)  
     - `tokens` (spans + features)  
     - `meta` (counts + IDs)

In [ ]:
# ---------------------------
# 3) Preprocess (fixed generator usage)
# ---------------------------
def preprocess_texts(
    texts_with_meta: List[Tuple[str, Dict[str, Any]]],
    do_clean: bool = True,
    do_langid: bool = True,
    do_sentences: bool = True,
    do_tokens: bool = True,
) -> List[Dict[str, Any]]:
    nlp = spacy.blank("xx")
    if do_sentences:
        nlp.add_pipe("sentencizer")

    results: List[Dict[str, Any]] = []

    if do_tokens or do_sentences:
        # Iterate with zip to consume the generator safely
        for (text, meta), doc in zip(texts_with_meta, nlp.pipe([t for t, _ in texts_with_meta])):
            text_clean = clean_text(text) if do_clean else text
            lang = detect_language(text) if do_langid else {"language": "xx", "score": None}

            # Sentences
            sents = []
            if do_sentences:
                for sid, s in enumerate(doc.sents):
                    sents.append({"id": sid, "start": s.start_char, "end": s.end_char, "text": s.text})

            # Tokens
            tokens = []
            if do_tokens:
                tok2sent = {}
                if do_sentences:
                    for sid, s in enumerate(doc.sents):
                        for tok in s:
                            tok2sent[tok.i] = sid
                for tok in doc:
                    tokens.append({
                        "id": tok.i,
                        "text": tok.text,
                        "start": tok.idx,
                        "end": tok.idx + len(tok.text),
                        "ws": tok.whitespace_ != "",
                        "is_punct": tok.is_punct,
                        "sent_id": tok2sent.get(tok.i) if do_sentences else None
                    })

            results.append({
                "text_original": text,
                "text_clean": text_clean,
                "language": lang,
                "sentences": sents,
                "tokens": tokens,
                "meta": {
                    **meta,
                    "char_count": len(text),
                    "token_count": len(tokens),
                    "sentence_count": len(sents),
                }
            })
    else:
        # No tokenization/sentences requested
        for text, meta in texts_with_meta:
            text_clean = clean_text(text) if do_clean else text
            lang = detect_language(text) if do_langid else {"language": "xx", "score": None}
            results.append({
                "text_original": text,
                "text_clean": text_clean,
                "language": lang,
                "sentences": [],
                "tokens": [],
                "meta": {
                    **meta,
                    "char_count": len(text),
                    "token_count": 0,
                    "sentence_count": 0,
                }
            })

    return results

In [ ]:
# call the preprocessing function and print the first results
pairs = [(r["text"], {"source": "CMA", "id": r["id"]}) for r in records]
out_api_id = preprocess_texts(pairs)
print(out_api_id[:10])

[{'text_original': '\ufeff-----------------------------------------------------------------------------\r\nMAG X870E TOMAHAWK WIFI (MS-7E59) V2.A83 BIOS Release\r\n-----------------------------------------------------------------------------\r\n\r\n1. This is AMI BIOS release\r\n\r\n2. This BIOS fixes the following problem of the previous version:\r\n-  Improve memory compatibility.\r\n-  Optimized Secure Erase mechanism.\r\n\r\n3. 2025/08/01\r\n\r\n\r\n[Below information is Traditional Chinese language]\r\n\r\nA. AMI BIOS 正式發行\r\n\r\nB. 此版本修正下列問題:\r\n-  改善記憶體相容性問題。\r\n-  優化安全性清除機制。\r\n\r\nC. 更新日期: 公元2025年8月1號\r\n\r\n\r\n[Below information is Simplified Chinese language]\r\n\r\nA. AMI BIOS 正式发行\r\n\r\nB. 此版本修正下列问题:\r\n-  改善内存兼容性问题。\r\n-  优化安全性清除机制。\r\n\r\nC. 更新日期: 公元2025年8月1号', 'text_clean': '\ufeff-----------------------------------------------------------------------------\r\nMAG X870E TOMAHAWK WIFI (MS-7E59) V2.A83 BIOS Release\r\n----------------------------------------------------

# Step 5: Put it all together

Call the functions. The code below provides three different options, depending on the types of calls, and it also includes an option that processes local examples.

Each option can be run independently and saves the output in a file within the Colab notebook.

In [74]:
# ---------------------------
# 4) MAIN (robust demos)
# ---------------------------
def save_records(records: List[Dict[str, str]], source: str = "CMA") -> None:
  if not records:
    raise ValueError("No records found (likely 404 if using the CMA API).")

  pairs = [(r["text"], {"source": source, "id": r["id"]}) for r in records]
  processed_texts = preprocess_texts(pairs)
  json_string = json.dumps(processed_texts, ensure_ascii=False, indent=2)
  # Use hash to prevent overwriting previous work
  hash = hashlib.sha1(json_string.encode("utf-8")).hexdigest()
  print(f"CMA by id: {json_string}")

  output_file = f"/content/output_{hash}.json"
  with open(output_file, "w", encoding="utf-8") as f:
    json.dump(out_api_id, f)
    print(f"Saved preprocessed files to {output_file}")

  if IN_COLAB:
    files.download(output_file)

# Save to
save_records(records)

CMA by id: [
  {
    "text_original": "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of their home at Argenteuil. Her face is rendered in a radically bold Impressionist technique of mere daubs of paint quickly applied, just as the snow and trees are defined by broad, broken strokes of pure white and green.",
    "text_clean": "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of their home at Argenteuil. Her face is rendered in a radically bold Impressionist technique of mere daubs of paint quickly applied, just as the snow and trees are defined by broad, broken strokes of pure white and green.",
    "language": {
      "language": "en",
      "score": -868.9007034301758
    },
    "sentences": [
      {
        "id": 0,
        "start": 0,
        "end": 130,
        "text": "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
    # OPTION B: CMA search (take a couple of hits, skip empty descriptions)
if __name__ == "__main__":
  try:
        records = fetch_cma("monet", mode="search", limit=5)
        pairs = [(r["text"], {"source": "CMA", "id": r["id"]}) for r in records if r["text"]]
        pairs = pairs[:2]  # first two with non-empty text
        if pairs:
            out_api_search = preprocess_texts(pairs)
            print("\nCMA search:")
            print(json.dumps(out_api_search, ensure_ascii=False, indent=2))
            # save the results in a local file
            output_file = "/content/output_api_search.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(out_api_search, f)
                print(f"Saved preprocessed files to {output_file}")
        else:
            print("\nCMA search: no descriptions returned for this query.")
  except Exception as e:
        print("API search failed:", e)


CMA search:
[
  {
    "text_original": "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of their home at Argenteuil. Her face is rendered in a radically bold Impressionist technique of mere daubs of paint quickly applied, just as the snow and trees are defined by broad, broken strokes of pure white and green.",
    "text_clean": "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors of their home at Argenteuil. Her face is rendered in a radically bold Impressionist technique of mere daubs of paint quickly applied, just as the snow and trees are defined by broad, broken strokes of pure white and green.",
    "language": {
      "language": "en",
      "score": -868.9007034301758
    },
    "sentences": [
      {
        "id": 0,
        "start": 0,
        "end": 130,
        "text": "This painting depicts Monet's first wife, Camille, outside on a snowy day passing by the French doors 

In [ ]:
    # OPTION C: Local examples
if __name__ == "__main__":
  examples = [
        ("Rome is the capital of Italy.", {"source": "local"}),
        ("Mark Rutte bezocht gisteren Groningen.", {"source": "local"})
    ]
  out_local = preprocess_texts(examples)
  print("\nLocal examples:")
  print(json.dumps(out_local, ensure_ascii=False, indent=2))

# save the results in a local file
output_file = "/content/output_local.json"
with open(output_file, "w", encoding="utf-8") as f:
  json.dump(out_local, f)
  print(f"Results saved in {output_file}")


Local examples:
[
  {
    "text_original": "Rome is the capital of Italy.",
    "text_clean": "Rome is the capital of Italy.",
    "language": {
      "language": "en",
      "score": -73.82194948196411
    },
    "sentences": [
      {
        "id": 0,
        "start": 0,
        "end": 29,
        "text": "Rome is the capital of Italy."
      }
    ],
    "tokens": [
      {
        "id": 0,
        "text": "Rome",
        "start": 0,
        "end": 4,
        "ws": true,
        "is_punct": false,
        "sent_id": 0
      },
      {
        "id": 1,
        "text": "is",
        "start": 5,
        "end": 7,
        "ws": true,
        "is_punct": false,
        "sent_id": 0
      },
      {
        "id": 2,
        "text": "the",
        "start": 8,
        "end": 11,
        "ws": true,
        "is_punct": false,
        "sent_id": 0
      },
      {
        "id": 3,
        "text": "capital",
        "start": 12,
        "end": 19,
        "ws": true,
        "is_punct": false

In [ ]:
# OPTION D: Local examples from a txt file (make sure to upload the txt file in the content folder of the Colab Notebook)
if __name__ == "__main__":
  # open a local .txt file as input. remember to change the path and filename to your file.
  with open("/content/input.txt", "r") as f:
    text = f.read()

    # then process the text of the input file
    examples = [(text, {"source": "local"})]
    out_local = preprocess_texts(examples)
    print("\nLocal examples:")
    print(json.dumps(out_local, ensure_ascii=False, indent=2))

# save the results in a local file
output_file = "/content/output_file.json"
with open(output_file, "w", encoding="utf-8") as f:
  json.dump(out_local, f)
  print(f"Results saved in {output_file}")


Local examples:
[
  {
    "text_original": "STRABO, the author of this work, was born at Amasia, or Amasijas, a town situated in the gorge of the mountains through which passes the river Iris, now the Ieschil Irmak, in Pontus, which he has described in the 12th book.[*] He lived during the reign of Augustus, and the earlier part of the reign of Tiberius; for in the 13th book[*] he relates how Sardes and other cities, which had suffered severely from earthquakes, had been repaired by the provident care of Tiberius the present Emperor; but the exact date of his birth, as also of his death, are subjects of conjecture only. Coraÿ and Groskurd conclude, though by a somewhat different argument, that he was born in the year B. C. 66, and the latter that he died A. D. 24. The date of his birth as argued by Groskurd, proceeds on the assumption that Strabo was in his thirty-eighth year when he went from Gyaros to Corinth, at which latter place Octavianus Caesar was then staying on his return to

## Save the results in a local .zip file

The output files are only saved locally in the Google Colab notebook, and will be deleted after the notebook is closed.

Two options are available:
1. If you are only interested in one of the files you generated, you can simply download the individual output file.
2. If you ran all options and want to save all output files, you can download all of them as a zip folder.


In [ ]:
# take the output_file generated by your chosen option and download it on your machine.
from google.colab import files
files.download("/content/output_file.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# take the output_file saved under content and download it locally as a .zip file
from google.colab import files
!zip -r /content/output_files.zip output_api_id.json output_api_search.json output_local.json output_file.json
files.download("/content/output_files.zip")

  adding: output_local.json (deflated 86%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>